In [34]:
%reload_ext autoreload
%autoreload 2
from main import *
from type import *
from utils import *
from algorithm import *
from test import *
import random

In [35]:
for k, v in types.items():
    print(f"{k} = {v}")

top = a -> a
one = (a -> b) -> a -> b
two = ((b -> c) /\ (a -> b)) -> a -> c
three = ((c -> d) /\ (b -> c) /\ (a -> b)) -> a -> d
omega1 = ((a -> b) /\ a) -> b
omega2 = ((a -> (b -> c)) /\ a /\ b) -> c
omega3 = ((b -> c) /\ (a -> b) /\ a) -> c
test1 = a -> (a -> b) -> b
test2 = ((b -> c) /\ a) -> (a -> b) -> c
test3 = ((a -> b -> c) /\ b) -> a -> c
test4 = ((a -> a) -> b) -> b
test5 = (((ω -> b) -> c) /\ ((a -> a) -> b)) -> c
test6 = (((a -> b) -> c) /\ ((ω -> a) -> b)) -> c
kcomb = a -> ω -> a
kastcomb = ω -> a -> a
ccomb = (a -> b -> c) -> b -> a -> c
bcomb = (b -> c) -> (a -> b) -> a -> c
scomb = (a -> c -> d) -> (b -> c) -> (a /\ b) -> d
sscomb = (b -> c -> d) -> ((a -> d -> e) /\ b) -> (a /\ c) -> e
ssscomb = ((a -> b -> f -> g) /\ (c -> d)) -> (a /\ (d -> e -> f) /\ c) -> (b /\ e) -> g
ucomb = (a -> e -> f) -> (b -> d -> e) -> (c -> d) -> (a /\ b /\ c) -> f
diagP = ((a -> b) /\ (c -> d)) -> (a /\ c) -> (b -> d -> e) -> e
zero = ω -> a -> a


In [53]:
t, s, r = deepType(4, 20, 4)
print(t)
print(r)

(((a2 -> a1) /\ (a0 -> a2) /\ (a1 -> a0)) -> a1 -> a1) /\ ((a3 -> a1 -> a2) -> (a2 -> a1) -> (a3 /\ a2) -> a2) /\ ((((a2 -> a3) -> a2) /\ ((ω -> a2) -> a3)) -> a2) /\ (((a0 -> a2) /\ a0) -> a2) /\ ((a0 -> a3) -> (a2 -> a0) -> a2 -> a3) /\ (a3 -> a3) /\ (((a3 -> a2) /\ a3) -> (a3 -> a3) -> a2) /\ (ω -> a0 -> a0) /\ (a1 -> ω -> a1) /\ (((a3 -> a3 -> a2) /\ a3) -> a3 -> a2) /\ (ω -> a3 -> a3) /\ ((((ω -> a2) -> a2) /\ ((a0 -> a0) -> a2)) -> a2) /\ ((((a2 -> a2) -> a0) /\ ((ω -> a2) -> a2)) -> a0) /\ (((a2 -> a3) /\ a2) -> (a2 -> a2) -> a3) /\ ((a3 -> a2) -> a3 -> a2) /\ (a3 -> ω -> a3) /\ (((a1 -> a3) /\ a2) -> (a2 -> a1) -> a3) /\ (ω -> a3 -> a3) /\ (a0 -> ω -> a0) /\ (((((a2 -> a0) /\ (a2 -> a2) /\ a2) -> a0) /\ ((a1 -> a3) -> a1 -> a3) /\ ((a2 -> a3) -> a2 -> a3) /\ (((a2 -> a3) /\ (a1 -> a2) /\ a1) -> a3) /\ ((((a1 -> a1) -> a1) /\ ((ω -> a1) -> a1)) -> a1) /\ ((a1 -> a0 -> a1) -> a0 -> a1 -> a1) /\ (a1 -> (a1 -> a0) -> a0) /\ (ω -> a3 -> a3) /\ (a0 -> ω -> a0) /\ (((a1 -> a0 -> a3 ->

In [59]:
t, s, r = manyVariables(7)
print(t)
print(r)

a12 /\ a5 /\ a2 /\ (a12 -> a12) /\ (ω -> a3 -> a3) /\ (a13 -> a13) /\ ((a7 /\ a13 /\ a1 /\ a7 /\ a9 /\ a8 /\ ((a4 /\ a5 /\ a4 /\ a8 /\ a12 /\ (ω -> a11 -> a11) /\ ((a9 /\ a2 /\ a3 /\ a1 /\ a3 /\ (((a0 -> a4) /\ (a6 -> a0) /\ (a6 -> a6)) -> a6 -> a4) /\ ((a0 /\ a4 /\ a4 /\ a10 /\ (((a0 -> a11) /\ (a1 -> a0) /\ a1) -> a11) /\ (((a13 -> a5) /\ a8) -> (a8 -> a13) -> a5) /\ ((a12 /\ a4 /\ a0 /\ (ω -> a0 -> a0) /\ (ω -> a0 -> a0) /\ (((a10 -> a7) /\ a10) -> a7) /\ ((a5 /\ a8 /\ a4 /\ a11 /\ a1 /\ (((a5 -> a2 -> a13 -> a0) /\ (a10 -> a6)) -> (a5 /\ (a6 -> a10 -> a13) /\ a10) -> (a2 /\ a10) -> a0) /\ (((((c -> d) /\ (b -> c) /\ (a -> b)) -> a -> d) /\ (a -> a) /\ (a -> ω -> a) /\ (((a -> b) /\ a) -> b) /\ (((b -> c) /\ a) -> (a -> b) -> c) /\ ((b -> c) -> (a -> b) -> a -> c) /\ (((a -> a) -> b) -> b)) -> a0)) -> a1)) -> a2)) -> a3)) -> a4)) -> a5)) -> a6)
{a -> a13, a0 -> d, a1 -> a5, a10 -> a4, a11 -> a6, a12 -> c, a13 -> a8, a2 -> a9, a3 -> a1, a4 -> a12, a5 -> a7, a6 -> a0, a7 -> a10, a8 ->

In [60]:
naiveComparison(t, s)

KeyboardInterrupt: 

In [61]:
%%timeit
comparePlain(t, s)

250 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%%timeit
compareSizeClasses(t, s)

29.8 ms ± 3.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
%%timeit
compareConstraintProp(t, s)

254 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%%timeit
compareConstraintPropSizeClasses(t, s)

5.59 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
t, s, r = manyVariables(10, 10)
print(t)
print(r)

a4 /\ a8 /\ a3 /\ a2 /\ a10 /\ a17 /\ a8 /\ (((a10 -> a8 -> a13) /\ a8) -> a10 -> a13) /\ (((a3 -> a11 -> a4) /\ a11) -> a3 -> a4) /\ ((a13 /\ a6 /\ a2 /\ a15 /\ a8 /\ a7 /\ (((a14 -> a5) /\ (a19 -> a14) /\ (a19 -> a19)) -> a19 -> a5) /\ (ω -> a4 -> a4) /\ ((((ω -> a5) -> a17) /\ ((a18 -> a18) -> a5)) -> a17) /\ ((a2 /\ a5 /\ a11 /\ a3 /\ a18 /\ a10 /\ a6 /\ a8 /\ (((a9 -> a12) /\ a18) -> (a18 -> a9) -> a12) /\ ((a8 /\ a15 /\ a14 /\ a8 /\ a16 /\ a10 /\ a14 /\ (((a14 -> a0 -> a12) /\ a0) -> a14 -> a12) /\ (((a0 -> a13 -> a5 -> a6) /\ (a18 -> a11)) -> (a0 /\ (a11 -> a5 -> a5) /\ a18) -> (a13 /\ a5) -> a6) /\ ((a17 /\ a4 /\ a18 /\ a9 /\ a3 /\ a6 /\ a2 /\ a6 /\ a5 /\ ((a6 /\ a0 /\ a17 /\ a8 /\ a19 /\ a6 /\ a10 /\ a4 /\ (a2 -> ω -> a2) /\ ((a11 /\ a13 /\ a9 /\ a16 /\ a11 /\ a5 /\ a10 /\ a12 /\ a6 /\ ((a14 /\ a3 /\ a8 /\ a13 /\ a0 /\ a4 /\ (((a7 -> a12) /\ (a18 -> a7) /\ a18) -> a12) /\ ((((ω -> a17) -> a8) /\ ((a18 -> a18) -> a17)) -> a8) /\ (((a13 -> a4) /\ (a17 -> a13)) -> (a13 /\ a17) ->

In [9]:
%%time
r1 = compareConstraintPropSizeClasses(t, s)
assert r1 == r

CPU times: user 16.6 ms, sys: 1.58 ms, total: 18.2 ms
Wall time: 18.5 ms


In [18]:
top = types["top"]

zero = types["zero"] 
test1 = types["test1"] 
one = types["one"]
two = types["two"]
three = types["three"]

x = Variable("x")
y = Variable("y")
z = Variable("z")
w = Variable("w")
v = Variable("v")
h = Variable("h")

d = Intersection([zero, test1, one, two, three])
d1 = Intersection([three, one, two, test1, zero])
# d1, r = randomRename(d1, [w, v, h, x, y, z])
d1, r = randomRename(d1)

# ssscom
print(typeSort(d))
print(naiveComparison(d, d1))
print(recomb(d, d1))
print(compareSizeClasses(d, d1))
print(compareConstraintProp(d, d1))
print(compareConstraintPropSizeClasses(d, d1))
print(r)

# random.seed(1000)
# t = types["bcomb"]
# print(t)
# s, r = randomRename(t)
# print(s)
# print(compareConstraintPropSizeClasses(t, s))
# print(r)

(a -> (a -> b) -> b) /\ ((a -> b) -> a -> b) /\ (ω -> a -> a) /\ (((a -> b) /\ (b -> c)) -> a -> c) /\ (((a -> b) /\ (b -> c) /\ (c -> d)) -> a -> d)
{a -> d, b -> c, c -> a, d -> b}
{a -> d, b -> c, c -> a, d -> b}
{a -> d, b -> c, c -> a, d -> b}
{a -> d, b -> c, c -> a, d -> b}
{a -> d, b -> c, c -> a, d -> b}
{a -> d, b -> c, c -> a, d -> b}


In [19]:
random.seed(1002)
ts = list(types.values())
ts = ts[:len(ts) - 1]
t = Intersection(list(ts))
random.shuffle(ts)
s, r = randomRename(Intersection(ts))
print(f"t = {t}")
print(f"s = {s}")
print(f"r = {r}")

t = (a -> a) /\ ((a -> b) -> a -> b) /\ (((b -> c) /\ (a -> b)) -> a -> c) /\ (((c -> d) /\ (b -> c) /\ (a -> b)) -> a -> d) /\ (((a -> b) /\ a) -> b) /\ (((a -> (b -> c)) /\ a /\ b) -> c) /\ (((b -> c) /\ (a -> b) /\ a) -> c) /\ (a -> (a -> b) -> b) /\ (((b -> c) /\ a) -> (a -> b) -> c) /\ (((a -> b -> c) /\ b) -> a -> c) /\ (((a -> a) -> b) -> b) /\ ((((ω -> b) -> c) /\ ((a -> a) -> b)) -> c) /\ ((((a -> b) -> c) /\ ((ω -> a) -> b)) -> c) /\ (a -> ω -> a) /\ (ω -> a -> a) /\ ((a -> b -> c) -> b -> a -> c) /\ ((b -> c) -> (a -> b) -> a -> c) /\ ((a -> c -> d) -> (b -> c) -> (a /\ b) -> d) /\ ((b -> c -> d) -> ((a -> d -> e) /\ b) -> (a /\ c) -> e) /\ (((a -> b -> f -> g) /\ (c -> d)) -> (a /\ (d -> e -> f) /\ c) -> (b /\ e) -> g) /\ ((a -> e -> f) -> (b -> d -> e) -> (c -> d) -> (a /\ b /\ c) -> f) /\ (((a -> b) /\ (c -> d)) -> (a /\ c) -> (b -> d -> e) -> e)
s = (((a -> (g -> d)) /\ a /\ g) -> d) /\ ((a -> b -> f) -> (g -> c -> b) -> (d -> c) -> (a /\ g /\ d) -> f) /\ (((a -> g) /\ (

In [16]:
%%timeit
r1 = naiveComparison(t, s)

CPU times: user 20 s, sys: 160 ms, total: 20.2 s
Wall time: 20.6 s


In [17]:
print(t)
print(s)

(a -> a) /\ ((a -> b) -> a -> b) /\ (((b -> c) /\ (a -> b)) -> a -> c) /\ (((c -> d) /\ (b -> c) /\ (a -> b)) -> a -> d) /\ (((a -> b) /\ a) -> b) /\ (((a -> (b -> c)) /\ a /\ b) -> c) /\ (((b -> c) /\ (a -> b) /\ a) -> c) /\ (a -> (a -> b) -> b) /\ (((b -> c) /\ a) -> (a -> b) -> c) /\ (((a -> b -> c) /\ b) -> a -> c) /\ (((a -> a) -> b) -> b) /\ ((((ω -> b) -> c) /\ ((a -> a) -> b)) -> c) /\ ((((a -> b) -> c) /\ ((ω -> a) -> b)) -> c) /\ (a -> ω -> a) /\ (ω -> a -> a) /\ ((a -> b -> c) -> b -> a -> c) /\ ((b -> c) -> (a -> b) -> a -> c) /\ ((a -> c -> d) -> (b -> c) -> (a /\ b) -> d) /\ ((b -> c -> d) -> ((a -> d -> e) /\ b) -> (a /\ c) -> e) /\ (((a -> b -> f -> g) /\ (c -> d)) -> (a /\ (d -> e -> f) /\ c) -> (b /\ e) -> g) /\ ((a -> e -> f) -> (b -> d -> e) -> (c -> d) -> (a /\ b /\ c) -> f) /\ (((a -> b) /\ (c -> d)) -> (a /\ c) -> (b -> d -> e) -> e)
(((e -> (b -> g)) /\ e /\ b) -> g) /\ ((e -> d -> f) -> (b -> a -> d) -> (g -> a) -> (e /\ b /\ g) -> f) /\ (((e -> b) /\ (g -> a))

In [20]:
%%time
r2 = recomb(t, s)

CPU times: user 15.5 ms, sys: 638 µs, total: 16.1 ms
Wall time: 16.9 ms


In [30]:
%%timeit
r3 = compareSizeClasses(t, s)

3.52 ms ± 54.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
%%time
r4 = compareConstraintProp(t, s)
print(r4)

KeyboardInterrupt: 

In [31]:
%%timeit
r5 = compareConstraintPropSizeClasses(t, s)

3.58 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
r1 = naiveComparison(t, s)
r2 = recomb(t, s)
r3 = compareSizeClasses(t, s)
# r4 = compareConstraintProp(t, s)
r5 = compareConstraintPropSizeClasses(t, s)
assert r == r1
assert r == r2
assert r == r3
# assert r == r4
assert r == r5

In [47]:
x = Variable("x")
y = Variable("y")
m = Variable("m")

t = typify([
    (a, [b, x]), (b, x)
])
s = typify([
    (m, [c, y]), (y, c)
])

print(recomb(t, s))
print(naiveComparison(t, s))
print(compareAC(t, s))
print(compareConstraintProp(t, s))

None
{a -> m, x -> c, b -> y}
{a -> m, b -> y, x -> c}
{a -> m, b -> y, x -> c}
